In [1]:
# ==> Psi4 & NumPy options, Geometry Definition <==
import numpy as np
import psi4

# Set numpy defaults
np.set_printoptions(precision=5, linewidth=200, suppress=True)

# Set Psi4 memory & output options
psi4.set_memory(int(2e9))
psi4.core.set_output_file('output.dat', False)

# Geometry specification
mol = psi4.geometry("""
O
H 1 0.96
H 1 0.96 2 104.5
symmetry c1
""")

# Psi4 options
psi4.set_options({'basis': 'aug-cc-pvdz',
                  'scf_type': 'direct',
                  'e_convergence': 1e-10,
                  'd_convergence': 1e-10})

wfn = psi4.core.Wavefunction.build(mol, psi4.core.get_global_option('basis'))


  Memory set to   1.863 GiB by Python driver.


In [2]:
aux = psi4.core.BasisSet.build(mol, "DF_BASIS_SCF", "", "RIFIT", "aug-cc-pVDZ")

In [3]:
# ==> Build Density-Fitted Integrals <==
# Get orbital basis & build zero basis
orb = wfn.basisset()
zero_bas = psi4.core.BasisSet.zero_ao_basis_set()

# Build instance of MintsHelper
mints = psi4.core.MintsHelper(orb)

# Build (P|pq) raw 3-index ERIs, dimension (1, Naux, nbf, nbf)
Ppq = mints.ao_eri(aux, zero_bas, orb, orb)

# Build & invert Coulomb metric, dimension (1, Naux, 1, Naux)
metric = mints.ao_eri(aux, zero_bas, aux, zero_bas)
metric.power(-0.5, 1.e-14)

# Remove excess dimensions of Ppq, & metric
Ppq = np.squeeze(Ppq)
metric = np.squeeze(metric)

# Build the Qso object
Qpq = np.einsum('QP,Ppq->Qpq', metric, Ppq, optimize=True)